In [203]:
import importlib 
import itertools
import collections

import pandas as pd

from climatools.lblnew import (setup_bestfit, 
                               setup_overlap,
                               pipeline, bestfit_params)

from IPython import display

importlib.reload(bestfit_params)

<module 'climatools.lblnew.bestfit_params' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/bestfit_params.py'>

In [204]:
def gas_names():
    return ('h2o', 'co2', 'o3', 'n2o', 'ch4')


def band_names():
    return ('1', '2', 
            '3a', '3b', '3c',
            '4', '5', '6', '7', '8', '9')


def bestfit_cases_byband():
    d = {}
    
    d['1'] = ['h2o']
    d['2'] = ['h2o']
    d['3a'] = ['h2o', 'co2', 'n2o']
    d['3b'] = ['h2o', 'co2']
    d['3c'] = ['h2o', 'co2']
    d['4'] = ['h2o']
    d['5'] = ['h2o', 'o3']
    d['6'] = ['h2o']
    d['7'] = ['h2o', 'n2o', 'ch4']
    d['8'] = ['h2o']
    d['9'] = ['h2o', 'co2', 'o3']
    return d


def overlap_cases_byband():
    d = collections.defaultdict(dict)
    
    d['7'][('h2o', 'n2o')] = {
        'atmpro': 'mls',
        'band': '7',
        'commitnumber': '000d800',
        'dv': 0.001,
        'molecule': {'h2o': 'atmpro', 'n2o': 3.2e-07},
        'nv': 1000,
        'tsfc': 294}
    
    d['5'][('h2o', 'o3')] = {
        'atmpro': 'mls',
        'band': '5',
        'commitnumber': '000d800',
        'dv': 0.001,
        'molecule': {'h2o': 'atmpro', 'o3': 'atmpro'},
        'nv': 1000,
        'tsfc': 294}
    
    d['9'][('h2o', 'co2')] = {
        'atmpro': 'mls',
        'band': '9',
        'commitnumber': '7c92840',
        'dv': 0.001,
        'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
        'nv': 1000,
        'tsfc': 294}
    
    return d



In [205]:
def get_hrefs_bestfit(molecule=None, band=None):
    atmpros = ('mls', 'saw', 'trp')
    
    hrefs = []
    for atmpro in atmpros:
        param = bestfit_params.kdist_params(molecule=molecule, 
                                            band=band)
        param['atmpro'] = atmpro
        param['option_compute_ktable'] = 0
        url = pipeline.nbviewer_url(param=param, setup=setup_bestfit)
        
        s = '''<a href="{}">{}</a>'''
        s = s.format(url, atmpro)
        hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs


def tb_hrefs_bestfit():
    df = pd.DataFrame(index=gas_names(), columns=band_names())

    for band, gass in bestfit_cases_byband().items():
        for gas in gass:
            df.loc[gas, band] = get_hrefs_bestfit(molecule=gas, 
                                                  band=band)
    
    df = df.fillna('')    
    return df


def get_hrefs_overlap(molecule=None, band=None):
    atmpros = ('mls', 'saw', 'trp')
    
    d = overlap_cases_byband()
    
    hrefs = []
    for atmpro in atmpros:
        param = d[band][molecule]
        param['atmpro'] = atmpro
        url = pipeline.nbviewer_url(param=param, setup=setup_overlap)
        
        s = '''<a href="{}">{}</a>'''
        s = s.format(url, atmpro)
        hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs


def tb_hrefs_overlap():
    which_bandgas = [('7', ('h2o', 'n2o')),
                     ('5', ('h2o', 'o3')),
                     ('9', ('h2o', 'co2'))]
    
    df = pd.DataFrame(columns=band_names())
    
    for band, molecule in which_bandgas:
        hrefs = get_hrefs_overlap(molecule=molecule, band=band)
        df.loc['+'.join(molecule), band] = hrefs
        
    df = df.fillna('')
    return df


In [206]:
def script():
    df_best = tb_hrefs_bestfit()
    df_over = tb_hrefs_overlap()
    
    df = pd.concat([df_best, df_over], 
                   axis=0, 
                   keys=['bestfit', 'overlap'])
    
    s_html = df.to_html(escape=False)
    display.display(display.HTML(s_html))

In [207]:
script()

In [208]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')